## Day 49 Proyect: LinkedIn Auto Apply Bot

### Proyecto Día 49 – LinkedIn Auto Apply Bot 💼🤖

Este proyecto automatiza el proceso de búsqueda y postulación de empleos en LinkedIn mediante Selenium.  
Utiliza el filtro "Easy Apply" para aplicar automáticamente a múltiples vacantes que requieren una sola interacción.

🚨 Requiere:
- Cuenta activa en LinkedIn (se recomienda una cuenta alterna)
- Configurar tu perfil con CV y preferencias de búsqueda
- Desactivar verificación en 2 pasos mientras usas Selenium

🔧 Herramientas aplicadas:
- Automatización con Selenium WebDriver
- Manejo de formularios y botones
- Excepciones y lógica condicional para evitar captchas o formularios complejos


#### Paso 1 – Login automático y acceso a la búsqueda

Usamos Selenium para abrir LinkedIn y loguearnos con el correo y contraseña proporcionados.


In [1]:
from selenium import webdriver                              # Libreria para controlar el navegador
from selenium.webdriver.common.by import By                 # Libreria para seleccionar elementos
from selenium.webdriver.common.keys import Keys             # Libreria para enviar acciones de teclado
import undetected_chromedriver as uc                        # Libreria para evitar detección de Selenium
import time                                                 # Libreria para manejar el tiempo
import os                                                   # Libreria para manejar el sistema operativo
from dotenv import load_dotenv                              # Libreria para cargar variables de entorno

from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

In [2]:
# Carga las variables del archivo .env
load_dotenv()  

True

In [3]:
# Datos de autenticación 
LINKEDIN_USER = os.environ["LINKEDIN_USER"]
LINKEDIN_PASSWORD = os.environ["LINKEDIN_PASSWORD"]

In [4]:
# Configurar opciones de Chrome
options = uc.ChromeOptions()                                           # Creamos una instancia de ChromeOptions para undetected_chromedriver
options.add_argument("--no-first-run")                                 # Prevenir configuraciones iniciales
options.add_argument("--no-default-browser-check")                     # Prevenir chequeo de navegador por defecto
options.add_argument("--disable-blink-features=AutomationControlled")  # Evitar detección de Selenium
options.add_argument("--disable-infobars")                             # Desactivar la barra de información
# options.add_argument("--start-maximized")                              # Opcional: abrir maximizado

# Importar módulos adicionales para espera y acciones
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Inicializar navegador con undetected-chromedriver
driver = uc.Chrome(options=options)

# Navegar a la página de inicio de sesión de LinkedIn
driver.get("https://www.linkedin.com/login")

# Inicializar WebDriverWait con un tiempo de espera de 10 segundos
wait = WebDriverWait(driver, 7)

#==========================================================================
#                  Ingresar credenciales de LinkedIn
#==========================================================================

# Esperar a que los campos de usuario y contraseña estén presentes
username_field = wait.until(EC.presence_of_element_located((By.ID, "username")))
password_field = wait.until(EC.presence_of_element_located((By.ID, "password")))
login_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@type='submit']")))
print("✅ Confirmado: Estamos en la página de login de LinkedIn")

# Ingresar las credenciales
username_field.send_keys(LINKEDIN_USER)           # Ingresar correo electrónico
password_field.send_keys(LINKEDIN_PASSWORD)       # Ingresar contraseña

# Dar click en el botón de login
driver.execute_script("arguments[0].scrollIntoView(true);", login_button)  # Desplazar la vista hacia el botón
driver.execute_script("arguments[0].click();", login_button)               # Hacer clic usando JavaScript para evitar problemas

print("🔑 Credenciales ingresadas y botón de login presionado")

# Verificar si el inicio de sesión fue exitoso (buscando un elemento que solo esté presente después del login)
try:
    perfil_elemento = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".global-nav__me")))
    print("✅ Inicio de sesión exitoso!\n")

    #==========================================================================
    #                  Navegar a la página de búsqueda de empleos
    #==========================================================================

    # Ir a la página de búsqueda de empleos con los filtros ya definidos en la URL
    print("Navegando a la búsqueda de empleos (con filtros ya aplicados)...")
    driver.get("https://www.linkedin.com/jobs/search/?f_AL=true&f_T=340%2C25190&f_WT=2&geoId=92000000&keywords=Data%20Scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=R&spellCorrectionEnabled=true")
    
    # Esperar a que la página cargue 
    print("⏳ Esperando a que cargue la página de resultados...")
    # time.sleep(5)
    
    # Encontrar las tarjetas de empleo
    try:
        # Usar el selector para encontrar las tarjetas de empleo
        resultados = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "li.ember-view.occludable-update.p0.relative.scaffold-layout__list-item")
        ))
        
        total_resultados = len(resultados)
        print(f"🔎 Se encontraron {total_resultados} empleos que coinciden con los criterios de búsqueda")
        
        # Seleccionar el indice de la solicitud a aplicar
        if total_resultados > 0:
            indice_seleccionado = 0                         # Seleccionar el indice de la solicitud a aplicar (0 para el primero)
            
            # Hacer clic en e la solicitud seleccionado
            print(f"\nSeleccionando la solicitud #{indice_seleccionado + 1} de {total_resultados}")
            solicutud = resultados[indice_seleccionado]                                # Seleccionar e la solicitud bajo el indice elegido
            driver.execute_script("arguments[0].scrollIntoView();", solicutud)         # Desplazar la vista hacia e la solicitud
            driver.execute_script("arguments[0].click();", solicutud)                  # Hacer clic usando JavaScript para evitar problemas
            
            # Buscar el título de la solicitud
            titulo_solicitud = wait.until(EC.presence_of_element_located(
                (By.CSS_SELECTOR, ".job-details-jobs-unified-top-card__job-title")
                )).text
            print(f"💼 Título de la solicitud: {titulo_solicitud}")

            #==========================================================================
            #                          Aplicar a la solicitud
            #==========================================================================
            
            # Buscar y hacer clic en el botón "Solicitud sencilla"
            boton_solicitud = wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, ".jobs-apply-button")
                ))
            driver.execute_script("arguments[0].scrollIntoView();", boton_solicitud)     # Desplazar la vista hacia el botón
            driver.execute_script("arguments[0].click();", boton_solicitud)              # Hacer clic usando JavaScript para evitar problemas
            
            print("🟢 Clic exitoso en el botón 'Solicitud sencilla'")
            
            # Esperar a que aparezca el modal de solicitud
            print("✅ Modal de solicitud abierto. El proceso se detiene aquí para revisión manual.")

        else:
            print("❌ No se encontraron empleos para aplicar.")
            
    except Exception as e:
        print(f"Error al buscar empleos: {e}")

except Exception as e:
    print("❌ No se pudo verificar el inicio de sesión exitoso.")
    print(f"Error: {e}")
    print(f"URL actual: {driver.current_url}")


✅ Confirmado: Estamos en la página de login de LinkedIn
🔑 Credenciales ingresadas y botón de login presionado
✅ Inicio de sesión exitoso!

Navegando a la búsqueda de empleos (con filtros ya aplicados)...
⏳ Esperando a que cargue la página de resultados...
🔎 Se encontraron 25 empleos que coinciden con los criterios de búsqueda

Seleccionando la solicitud #1 de 25
💼 Título de la solicitud: Domo BI Analyst (Req. English)
🟢 Clic exitoso en el botón 'Solicitud sencilla'
✅ Modal de solicitud abierto. El proceso se detiene aquí para revisión manual.


#### 🧠 Conclusión

Este proyecto integró scraping, automatización y control de formularios en una interfaz real:

- 🔒 Login automático seguro
- 🧭 Navegación por elementos dinámicos
- 🤖 Lógica para filtrar formularios simples y evitar errores
- 📩 Automatización de postulaciones múltiples